In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cartopy.crs as ccrs
import cartopy
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import xarray as xr

/home/harry/miniconda3/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
plt.rcParams['figure.facecolor']='white'
plt.rcParams['axes.facecolor']='white'

In [4]:
# to do: select only the applicable date from L4 
#    and try restrict lats and lons of l2 dataset

dc = xr.open_dataset('../Data/cci_sst.nc')
l2 = xr.open_dataset('../Data/ESA_CCI_L2P_AVHRRMTA_G_20081201.nc')

In [10]:
dc['analysed_sst']

array([-19.831053, -19.69238 , -19.733397, ..., -53.607418, -53.560543,
       -53.650387], dtype=float32)

In [11]:
# to do: make sure the sst data in line 26 is only adding the data for the index of interest
#    and consider saving only the SST value in order to plot only the difference in point
#    data of L2.

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


te = []

lon1 = []
lat1 = []

# dc = "your data"

latj = l2['latitude'] 
lonj = l2['longitude']
for i in range(0, len(lonj)):
    cc = dc.sel(lat=slice(find_nearest(dc['lat'], latj[i].data),
                               find_nearest (dc['lat'], latj[i].data)),
                lon=slice(find_nearest(dc['lon'], lonj[i].data),
                                find_nearest(dc['lon'], lonj[i].data)))
    "the variable you want"
    temp = cc['analysed_sst'].data
    te.append(temp)

    lon1.append(cc['lon'])
    lat1.append(cc['lat'])

KeyboardInterrupt: 